In [1]:
import streamlit as st
import pandas as pd
import joblib
import matplotlib.pyplot as plt

# Configuration de la page
st.set_page_config(page_title="Application NOx", layout="wide")

st.title("Prédiction de la pollution NOx")
st.markdown("Cette application utilise deux modèles pour prédire les niveaux de NOx et détecter les alertes.")

# 1. Téléchargement des données par l'utilisateur
st.subheader("📁 Téléchargez vos données")
uploaded_file = st.file_uploader("Choisissez un fichier CSV", type="csv")

if uploaded_file:
    # Chargement des données depuis le fichier téléchargé avec gestion des valeurs manquantes
    try:
        df = pd.read_csv(uploaded_file, na_values=["null", "NaN", "NA"])
        # Conversion de la colonne 'date' en datetime
        df['date'] = pd.to_datetime(df['date'], format="%d.%m.%Y %H:%M")
    except Exception as e:
        st.error(f"Erreur lors du chargement des données : {e}")
        st.stop()

    st.write("Voici un aperçu des données :")
    st.dataframe(df.head())

    # 2. Préparation des features
    features = df.drop(['date', 'Nox_baf', 'Nox opsis'], axis=1)
    features = features.apply(pd.to_numeric, errors='coerce')
    features = features.fillna(features.mean())

    # 3. Chargement des modèles
    @st.cache_resource
    def load_models():
        model_baf = joblib.load("Nox1_modèle.pkl")
        model_opsis = joblib.load("Nox_opsis_linearregression.pkl")
        return model_baf, model_opsis

    model_baf, model_opsis = load_models()

    # 4. Prédictions
    df['Nox_baf_pred'] = model_baf.predict(features)
    df['Nox_opsis_pred'] = model_opsis.predict(features)

    # 5. Fonctions d’alerte
    def alerte_baf(val):
        if val >= 500:
            return "DANGER"
        elif val >= 400:
            return "ATTENTION"
        else:
            return "OK"

    def alerte_opsis(val):
        if val >= 450:
            return "DANGER"
        elif val >= 350:
            return "ATTENTION"
        else:
            return "OK"

    df['Alerte_baf'] = df['Nox_baf_pred'].apply(alerte_baf)
    df['Alerte_opsis'] = df['Nox_opsis_pred'].apply(alerte_opsis)

    # 6. Affichage des alertes sous forme de graphique
    st.subheader("Distribution des alertes")

    alert_baf_counts = df['Alerte_baf'].value_counts()
    alert_opsis_counts = df['Alerte_opsis'].value_counts()

    # Graphiques à barres
    fig_alert_baf, ax_alert_baf = plt.subplots()
    ax_alert_baf.bar(alert_baf_counts.index, alert_baf_counts.values, color=['green', 'orange', 'red'])
    ax_alert_baf.set_title("Distribution des alertes NOx BAF")
    ax_alert_baf.set_xlabel("Alerte")
    ax_alert_baf.set_ylabel("Nombre d'occurrences")
    st.pyplot(fig_alert_baf)

    fig_alert_opsis, ax_alert_opsis = plt.subplots()
    ax_alert_opsis.bar(alert_opsis_counts.index, alert_opsis_counts.values, color=['green', 'orange', 'red'])
    ax_alert_opsis.set_title("Distribution des alertes NOx OPSIS")
    ax_alert_opsis.set_xlabel("Alerte")
    ax_alert_opsis.set_ylabel("Nombre d'occurrences")
    st.pyplot(fig_alert_opsis)

    # 7. Affichage interactif des prédictions avec filtre par date
    st.subheader("📈 Visualisation des prédictions")

    target_option = st.selectbox("Choisissez le type de NOx à afficher :", ["Nox_baf", "Nox_opsis"])

    # Sélection de la plage de dates
    start_date, end_date = st.date_input(
        "Sélectionner la plage de dates",
        [df['date'].min(), df['date'].max()],
        min_value=df['date'].min(),
        max_value=df['date'].max()
    )

    # Filtrer les données selon la plage de dates
    df_filtered = df[(df['date'] >= pd.to_datetime(start_date)) & (df['date'] <= pd.to_datetime(end_date))]

    if target_option == "Nox_baf":
        col_pred = "Nox_baf_pred"
        col_alert = "Alerte_baf"
        seuils = [500, 400]
    else:
        col_pred = "Nox_opsis_pred"
        col_alert = "Alerte_opsis"
        seuils = [450, 350]

    colors = {"OK": "green", "ATTENTION": "orange", "DANGER": "red"}

    fig, ax = plt.subplots(figsize=(14, 6))

    for alrt, color in colors.items():
        subset = df_filtered[df_filtered[col_alert] == alrt]
        ax.scatter(subset['date'], subset[col_pred], label=alrt, color=color, s=10)

    for seuil in seuils:
        ax.axhline(y=seuil, color='red' if seuil == seuils[0] else 'orange', linestyle='--', linewidth=1.5, label=f'Seuil {seuil}')

    ax.set_title(f"Prédictions et alertes pour {target_option}")
    ax.set_xlabel("Date")
    ax.set_ylabel(f"{target_option} prédit")
    ax.grid(True)
    ax.legend()
    st.pyplot(fig)

    # 8. Téléchargement des résultats sous forme de CSV
    st.subheader("📥 Télécharger les résultats")

    @st.cache_data
    def convert_df(df):
        return df.to_csv(index=False).encode('utf-8')

    csv = convert_df(df[['date', 'Nox_baf_pred', 'Nox_opsis_pred', 'Alerte_baf', 'Alerte_opsis']])

    st.download_button(
        label="Télécharger les résultats",
        data=csv,
        file_name='resultats_predictions.csv',
        mime='text/csv'
    )


2025-05-15 10:15:13.568 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 10:15:13.570 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 10:15:15.408 
  command:

    streamlit run C:\Users\User\anaconda3\envs\TF\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-15 10:15:15.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 10:15:15.416 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 10:15:15.420 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 10:15:15.429 No runtime found, using MemoryCacheStorageManager
2025-05-15 10:15:15.438 No 